# 응급_법정동_행정동_변환
- https://www.data.go.kr/data/15077871/openapi.do
- 2021 년도 이후의 데이터는 모두 제거
- 하나의 행정동에 법정동이 여러개일 경우, 합친다

In [1]:
import pandas as pd

In [44]:
dong_info = pd.read_excel('./externals/jscode20211231/KIKmix.20211231.xlsx') #.drop(columns=['시도명', '행정동코드', '법정동코드'])
dong_info

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN
...,...,...,...,...,...,...,...,...
21745,5013061000,제주특별자치도,서귀포시,중문동,5013011400,대포동,20060701,NaN
21746,5013061000,제주특별자치도,서귀포시,중문동,5013011800,하원동,20060701,NaN
21747,5013062000,제주특별자치도,서귀포시,예래동,5013011900,색달동,20060701,NaN
21748,5013062000,제주특별자치도,서귀포시,예래동,5013012000,상예동,20060701,NaN


In [47]:
dong_info = dong_info.loc[dong_info['시도명'] == '대구광역시']
dong_info = dong_info.drop(columns=['행정동코드', '법정동코드', '말소일자']).loc[~pd.isna(dong_info['시군구명'])].loc[~pd.isna(dong_info['읍면동명'])]

In [48]:
dong_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395 entries, 1149 to 1550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도명     395 non-null    object
 1   시군구명    395 non-null    object
 2   읍면동명    395 non-null    object
 3   동리명     395 non-null    object
 4   생성일자    395 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 18.5+ KB


In [49]:
dong_info['생성일자'] = dong_info['생성일자'].astype(str).str[:4]
dong_info

,시도명,시군구명,읍면동명,동리명,생성일자
1149,대구광역시,중구,동인동,동인동1가,2011
1150,대구광역시,중구,동인동,동인동2가,2011
1151,대구광역시,중구,동인동,동인동3가,2011
1152,대구광역시,중구,동인동,동인동4가,2011
1153,대구광역시,중구,삼덕동,삼덕동1가,1998
...,...,...,...,...,...
1546,대구광역시,달성군,구지면,화산리,1995
1547,대구광역시,달성군,구지면,수리리,1995
1548,대구광역시,달성군,구지면,징리,1995
1549,대구광역시,달성군,구지면,오설리,1995


In [50]:
dong_info = dong_info.drop(dong_info.loc[dong_info['생성일자'].astype(int) > 2021].index).loc[~pd.isna(dong_info['시군구명'])].loc[~pd.isna(dong_info['읍면동명'])].reset_index(drop=True)
dong_info

,시도명,시군구명,읍면동명,동리명,생성일자
0,대구광역시,중구,동인동,동인동1가,2011
1,대구광역시,중구,동인동,동인동2가,2011
2,대구광역시,중구,동인동,동인동3가,2011
3,대구광역시,중구,동인동,동인동4가,2011
4,대구광역시,중구,삼덕동,삼덕동1가,1998
...,...,...,...,...,...
390,대구광역시,달성군,구지면,화산리,1995
391,대구광역시,달성군,구지면,수리리,1995
392,대구광역시,달성군,구지면,징리,1995
393,대구광역시,달성군,구지면,오설리,1995


In [51]:
dong_cnt = dong_info.drop(columns=['시도명']).groupby(['시군구명', '읍면동명']).count().reset_index()
dong_cnt

,시군구명,읍면동명,동리명,생성일자
0,남구,대명10동,1,1
1,남구,대명11동,1,1
2,남구,대명1동,1,1
3,남구,대명2동,1,1
4,남구,대명3동,1,1
...,...,...,...,...
139,중구,동인동,4,4
140,중구,삼덕동,3,3
141,중구,성내1동,18,18
142,중구,성내2동,29,29


In [52]:
one_h_many_d = dong_cnt.loc[dong_cnt['동리명']>1]['읍면동명'].values
one_h_many_d

array(['송현2동', '신당동', '용산1동', '용산2동', '월성1동', '월성2동', '유천동', '이곡1동',
       '이곡2동', '장기동', '죽전동', '진천동', '가창면', '구지면', '논공읍', '논공읍공단출장소',
       '다사읍', '옥포읍', '유가읍', '하빈면', '현풍읍', '화원읍', '공산동', '도평동', '동촌동',
       '불로.봉무동', '신암2동', '안심1동', '안심2동', '안심3동', '안심4동', '지저동', '해안동',
       '혁신동', '고성동', '관문동', '국우동', '노원동', '무태조야동', '칠성동', '상중이동', '원대동',
       '고산1동', '고산2동', '고산3동', '수성2.3가동', '대신동', '동인동', '삼덕동', '성내1동',
       '성내2동', '성내3동'], dtype=object)

In [54]:
dong_info.drop(columns=['시군구명']).loc[dong_info['읍면동명'].isin(one_h_many_d)].to_csv('./externals/법정동_행정동_중복_mapper.csv')

In [56]:
dong_info.drop(columns=['시군구명']).to_csv('./externals/법정동_행정동_mapper.csv')